<a href="https://colab.research.google.com/github/Fush1m1/colab_copies/blob/main/signate220319beginer%E9%99%90%E5%AE%9A%E3%82%B3%E3%83%B3%E3%83%9A%EF%BC%88%E7%B3%96%E5%B0%BF%E7%97%85%EF%BC%89220326_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データとライブラリの読み込み

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submit.csv',header=None)

In [ ]:
train

In [ ]:
test

In [18]:
sub

,0,1
0,398,0
1,3833,0
2,4836,0
3,4572,0
4,636,0
...,...,...
1995,3138,0
1996,191,0
1997,3294,0
1998,3073,0


# クレンジングと特徴量の作成

In [19]:
train = train.drop("index", axis=1)
test = test.drop("index", axis=1)

In [ ]:
train.isnull()

# モデル構築

In [4]:
X_train = train.drop(['Outcome'], axis=1)
y_train = train['Outcome']

## LightGBM

In [8]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

### ホールドアウト法

In [ ]:
from sklearn.model_selection import train_test_split

#X_trainとy_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

#LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')

#trainとvalidを指定し学習
gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
        early_stopping_rounds=20, #20回連続でlossが下がらなかったら終了
        verbose=10  #10round毎に、lossを表示
)


In [ ]:
#valid_xについて推論
#out of fold　=　学習に使っているデータ以外の予測
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
#正解率
print('score', round(accuracy_score(valid_y, oof)*100,2), '%')

score 78.79 %


### optuna

In [ ]:
!pip3 install optuna

In [8]:
import optuna
import optuna.integration.lightgbm as lgb

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#X_trainとy_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train, test_size=0.33, random_state=0)

#LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')

#trainとvalidを指定し学習
gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
        early_stopping_rounds=20, #20回連続でlossが下がらなかったら終了
        verbose=10  #10round毎に、lossを表示
)

Training until validation scores don't improve for 20 rounds.
[10]	valid_0's binary_logloss: 0.490868
[20]	valid_0's binary_logloss: 0.473108
[30]	valid_0's binary_logloss: 0.46972
[40]	valid_0's binary_logloss: 0.468288
[50]	valid_0's binary_logloss: 0.467052
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.466403


LGBMClassifier(objective='binary')

In [19]:
#valid_xについて推論
#out of fold　=　学習に使っているデータ以外の予測
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
#正解率
print('score', round(accuracy_score(valid_y, oof)*100,2), '%')

score 78.79 %


### 分割交差検証での学習

In [9]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)  # 3分割交差検証のためにインスタンス化

# スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, y_train)):
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = y_train[train_index]
    valid_y = y_train[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary')
    gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                early_stopping_rounds=20,
                verbose= -1) # 学習の状況を表示しない
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list), "%")  

fold1 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[56]	valid_0's binary_logloss: 0.401825
fold1 end

fold2 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.430412
fold2 end

fold3 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[31]	valid_0's binary_logloss: 0.451109
fold3 end

[81.1, 79.8, 78.4] 平均score 79.76666666666667 %


### グリッドサーチ

In [ ]:
# GridSearchCVをimport
from sklearn.model_selection import GridSearchCV

gbm = lgb.LGBMClassifier(objective='binary')

# 試行するパラメータを羅列する
params = {
    'max_depth': [2, 3, 4, 5],
    'reg_alpha': [0, 1, 10, 100],
    'reg_lambda': [0, 1, 10, 100],
}

grid_search = GridSearchCV(
                           gbm,  # 分類器を渡す
                           param_grid=params,  # 試行してほしいパラメータを渡す
                           cv=3,  # 3分割交差検証でスコアを確認
                          )

grid_search.fit(X_train, Y_train)  # データを渡す

print(grid_search.best_score_)  # ベストスコアを表示
print(grid_search.best_params_)  # ベストスコアのパラメータを表示

# 提出

In [20]:
#testの予測
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)
sub[1] = test_pred  # sample_submitの列1をtest_predに置き換え

In [21]:
sub

,0,1
0,398,0
1,3833,0
2,4836,0
3,4572,0
4,636,0
...,...,...
1995,3138,0
1996,191,0
1997,3294,1
1998,3073,0


In [22]:
sub.to_csv('220326002.csv', index=False)